In [1]:
import pandas as pd
import numpy as np
import re

desired_width=320
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns',20)
pd.set_option('display.max_rows', 500)


# Load language model
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

c:\Users\poseidon\anaconda\envs\NLP\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0



# NER - Named Entity Recognition


In [2]:
doc=nlp('''The Slim Fit MVP is a slim fitting, ultra comfortable classic Jeans. 
The Extreme Motion series are the ultimate combination between extreme comfort, freedom to move 
and authentic denim, coming here in a slim fit execution, very much like Guess.
Featuring the amazing comfort, movement and unbeatable stretch of Lee’s Extreme Motion collection,
these slim fit jeans are made with motion in mind. Look for the grey elastic in the super comfy athletic
waistband and the signature donut button and brown leather back patch. Lee jeans are sold in the same
stores as Wrangler. Made using recycled fabric and constructed using a cleverly tapered slim fit.''')

for entity in doc.ents:
      print(entity.label_, ' | ', entity.text)
displacy.render(doc, style="ent")

PERSON  |  Slim Fit MVP
NORP  |  Jeans
WORK_OF_ART  |  The Extreme Motion
PERSON  |  Guess
ORG  |  Lee’s Extreme Motion
PERSON  |  Lee
PRODUCT  |  Wrangler


# Customize NER Pipeline

In [5]:
# Import data
brands=pd.read_csv("Data/brands.csv")
jeans=pd.read_csv("Data/jeans.csv")["name"]

In [6]:
# training data
TRAIN_DATA = [( jeans[x] , {"entities": [(0, len(jeans[x].split(" ")[0] ), "BRAND")]}) for x in jeans.index]
TRAIN_DATA[10:20]

[('Lee Slim Fit Jeans - Rinse', {'entities': [(0, 3, 'BRAND')]}),
 ('Lee Slim Fit Jeans - Aristocrat', {'entities': [(0, 3, 'BRAND')]}),
 ('Lee Regular Straight Fit Jeans - Black', {'entities': [(0, 3, 'BRAND')]}),
 ('Lee Regular Straight Fit Jeans - Cougar', {'entities': [(0, 3, 'BRAND')]}),
 ('Lee Regular Straight Fit Jeans - Trip', {'entities': [(0, 3, 'BRAND')]}),
 ('Lee Regular Straight Fit Jeans - Maddox', {'entities': [(0, 3, 'BRAND')]}),
 ('Columbia Triple Canyon Trousers - Black', {'entities': [(0, 8, 'BRAND')]}),
 ('Wrangler Texas Regular Jeans - Darkstone', {'entities': [(0, 8, 'BRAND')]}),
 ('Wrangler Texas Stretch Regular Jeans - Worn Broke',
  {'entities': [(0, 8, 'BRAND')]}),
 ('Wrangler Texas Stretch Regular Jeans - Blue Black',
  {'entities': [(0, 8, 'BRAND')]})]

In [7]:
# Load pre-existing spacy model
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random

nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

# New label to add
LABEL = "BRAND"

# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]



# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :
    sizes = compounding(1.0, 4.0, 1.001)
    # Training for 30 iterations     
    for itn in range(30):
    # shuffle examples before training
        random.shuffle(TRAIN_DATA)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=sizes)
        # dictionary to store losses
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            example = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example.append(Example.from_dict(doc, annotations))
                nlp.update(
                            example,
                            drop=0.5,  # dropout - make it harder to memorise data
                            losses=losses,
                        )
            print("Losses", losses)
    

Losses {'ner': 3.7635408667420336}
Losses {'ner': 6.675213562864002}
Losses {'ner': 9.408136443570612}
Losses {'ner': 10.962251800717269}
Losses {'ner': 15.363009546627016}
Losses {'ner': 17.26915841605238}
Losses {'ner': 19.81764139945709}
Losses {'ner': 21.784123097277543}
Losses {'ner': 23.08165434605204}
Losses {'ner': 24.963745585455094}
Losses {'ner': 27.067796868771236}
Losses {'ner': 28.599868178328304}
Losses {'ner': 30.167177096397772}
Losses {'ner': 32.50381999231395}
Losses {'ner': 33.886406404418224}
Losses {'ner': 35.795497635784116}
Losses {'ner': 37.55137706896838}
Losses {'ner': 39.09881192143617}
Losses {'ner': 41.08661768775266}
Losses {'ner': 43.07333696149734}
Losses {'ner': 45.034794664390084}
Losses {'ner': 46.74684492209072}
Losses {'ner': 48.247573403063065}
Losses {'ner': 49.97132759019397}
Losses {'ner': 51.397836263820786}
Losses {'ner': 52.70774170769191}
Losses {'ner': 53.95083438127652}
Losses {'ner': 55.727286220987374}
Losses {'ner': 57.66922924153693}


In [8]:
doc=nlp('''The Slim Fit MVP is a slim fitting, ultra comfortable classic Jeans. 
The Extreme Motion series are the ultimate combination between extreme comfort, freedom to move 
and authentic denim, coming here in a slim fit execution, very much like Guess.
Featuring the amazing comfort, movement and unbeatable stretch of Lee’s Extreme Motion collection,
these slim fit jeans are made with motion in mind. Look for the grey elastic in the super comfy athletic
waistband and the signature donut button and brown leather back patch. Lee jeans are sold in the same
stores as Wrangler. Made using recycled fabric and constructed using a cleverly tapered slim fit.''')

for entity in doc.ents:
      print(entity.label_, ' | ', entity.text)
displacy.render(doc, style="ent")

BRAND  |  The
BRAND  |  ultra
BRAND  |  Guess.
BRAND  |  Lee
BRAND  |  Lee
BRAND  |  Wrangler
